## Setting Up

### Installing Dependencies

In [9]:
%pip install openai pydantic instructor pandas python-dotenv
from IPython.display import clear_output ; clear_output()

### Loading Azure Open AI configuration

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")

from openai import AzureOpenAI

aoai = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

## Using Structured Input/Output with OpenAI

In [11]:
import instructor
from pydantic import BaseModel, Field
from typing import List
from enum import Enum
import json
from pandas import DataFrame

client = instructor.from_openai(aoai)

def print_json(data):
    print(json.dumps(data, indent=2))

def print_schema(model: BaseModel):
    print_json(model.model_json_schema(), indent=2)

def print_result(result: BaseModel):
    print(result.model_dump_json(indent=2))

def json_dataframe(data):
    return DataFrame([item.dict() for item in data])

def llm(response_model: BaseModel = BaseModel, system: str = None, user: str = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    if user:
        messages.append({"role": "user", "content": user})
    result = client.chat.completions.create(
        model=GPT_4_O_MODEL_NAME,
        response_model=response_model,
        messages=messages,
    )
    return result

## Working with Text

### Translation and Normalization

In [4]:
class TranslatedString(BaseModel):
    input_language: str = Field(..., description="The language of the original text, as 2-letter language code.")
    translation: str

translation = llm(
    TranslatedString,
    "Detect the language of the original text and translate it into English.",
    "Sprachkenntnisse sind ein wichtiger Bestandteil der Kommunikation.",
)

print_result(translation)

{
  "input_language": "de",
  "translation": "Language skills are an important part of communication."
}


In [18]:
class Translation(BaseModel):
    language: str = Field(..., description="The language of the translated text, as 2-letter language code.")
    translation: str

class Translations(BaseModel):
    translations: List[Translation]

target_languages = ["de", "fr", "it", "es", "he"]

translations = llm(
    Translations,
    f"Translate the user-provided text into the following languages: {json.dumps(target_languages)}",
    "Large Language Models are a powerful tool for natural language processing.",
)

print_result(translations)

{
  "translations": [
    {
      "language": "de",
      "translation": "Große Sprachmodelle sind ein mächtiges Werkzeug für die Verarbeitung natürlicher Sprache."
    },
    {
      "language": "fr",
      "translation": "Les grands modèles de langage sont un outil puissant pour le traitement du langage naturel."
    },
    {
      "language": "it",
      "translation": "I modelli linguistici di grandi dimensioni sono uno strumento potente per l'elaborazione del linguaggio naturale."
    },
    {
      "language": "es",
      "translation": "Los modelos de lenguaje grandes son una herramienta poderosa para el procesamiento del lenguaje natural."
    },
    {
      "language": "he",
      "translation": "מודלים של שפה גדולה הם כלי חזק לעיבוד שפה טבעית."
    }
  ]
}


In [6]:

text_styles = ["formal", "informal", "casual", "academic", "professional", "business"]

class NormalizedText(BaseModel):
    style: str = Field(
        ...,
        description=(f"The style of the text normalization.  Options: {json.dumps(text_styles)}"))
    text: str

class NormalizedTexts(BaseModel):
    normalized_texts: List[NormalizedText]

normalizations = llm(
    NormalizedTexts,
    f"Normalize the user-provided text into the following styles: {json.dumps(text_styles)}",
    "Large Language Models are a powerful tool for natural language processing.",
)

print_result(normalizations)

{
  "normalized_texts": [
    {
      "style": "formal",
      "text": "Large Language Models represent a potent instrument for natural language processing."
    },
    {
      "style": "informal",
      "text": "Big language models are a super handy tool for handling natural language."
    },
    {
      "style": "casual",
      "text": "Large Language Models are pretty awesome for working with natural language."
    },
    {
      "style": "academic",
      "text": "Large Language Models constitute a significant advancement in the field of natural language processing, providing substantial capabilities for a variety of applications."
    },
    {
      "style": "professional",
      "text": "Large Language Models are a powerful asset for natural language processing tasks."
    },
    {
      "style": "business",
      "text": "Large Language Models offer a robust solution for enhancing natural language processing capabilities, making them a valuable asset in business applications."
 

### Unstructured Data

In [7]:
class AddressInfo(BaseModel):
    first_name: str
    last_name: str
    street: str
    house_number: str
    postal_code: str
    city: str
    state: str
    country: str

address_info = llm(
    AddressInfo,
    "Eleanor Berger lives in Switzerland, in the canton of Zürich. Her home is at Freilagerstrasse 97, 8047 Zürich."
)

print_result(address_info)

{
  "first_name": "Eleanor",
  "last_name": "Berger",
  "street": "Freilagerstrasse",
  "house_number": "97",
  "postal_code": "8047",
  "city": "Zürich",
  "state": "Zürich",
  "country": "Switzerland"
}


In [24]:
input_text = (
  "During my recent travels, I had the pleasure of visiting several fascinating locations. "
  "My journey began at the office of Dr. Elena Martinez, 142B Elm Street, San Francisco, "
  "CA 94107, USA. Her office, nestled in the bustling heart of the city, was a hub of "
  "innovation and creativity. Next, I made my way to the historic residence of Mr. Hans "
  "Gruber located at 3. Stock, Goethestrasse 22, 8001 Zürich, Schweiz. The old building, "
  "with its classic Swiss architecture, stood as a testament to the city’s rich cultural "
  "heritage. My adventure continued at the tranquil countryside home of Satoshi Nakamoto, "
  "2-15-5, Sakura-cho, Musashino-shi, Tokyo-to 180-0003, Japan. Their home was surrounded by "
  "beautiful cherry blossoms, creating a picturesque scene straight out of a postcard. In "
  "Europe, I visited the charming villa of Mme. Catherine Dubois, 15 Rue de la République, "
  "69002 Lyon, France. The cobblestone streets and historic buildings of Lyon provided a "
  "perfect backdrop to her elegant home. Finally, my journey concluded at the modern apartment "
  "of Mr. David Johnson, Apt 7B, 34 Queen Street, Toronto, ON M5H 2Y4, Canada. The sleek "
  "design of the apartment building mirrored the contemporary vibe of the city itself."
  )

class Addresses(BaseModel):
    addresses: List[AddressInfo]

addresses_in_text = llm(
    Addresses,
    "Return all the addresses in the user-provided text.",
    input_text,
)

json_dataframe(addresses_in_text.addresses)

,first_name,last_name,street,house_number,postal_code,city,state,country
0,Elena,Martinez,Elm Street,142B,94107,San Francisco,CA,USA
1,Hans,Gruber,Goethestrasse,22,8001,Zürich,,Schweiz
2,Satoshi,Nakamoto,Sakura-cho,2-15-5,180-0003,Musashino-shi,Tokyo-to,Japan
3,Catherine,Dubois,Rue de la République,15,69002,Lyon,,France
4,David,Johnson,Queen Street,34,M5H 2Y4,Toronto,ON,Canada


## Decision Making

## Planning and Tool-Use